In [8]:
PROCESSES_COUNT = 20

print("Importing...")
from multiprocessing import Pool

import importlib
import os
import sys
TOOLS_PATH = f"{os.getenv('MASTER_PROJECT_ROOT_FOLDER')}/logistics"
sys.path.insert(0, TOOLS_PATH)
from tools import run_tools, plot_tools, data_tools, model_tools
import seaborn as sns
import openmc
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# from progressbar import progressbar
from tqdm import tqdm
from uncertainties import ufloat

results = []

result_folders = run_tools.get_all_result_dirs()
print(f"Found {len(result_folders)} result folders")

def handle_folder(result_folder):
    result_path = f"../results/{result_folder}"
    run_settings = run_tools.load_run_settings_json(result_path)

    statepoint_number = run_settings["active_batch_count"] + run_settings["inactive_batch_count"]
    sp = openmc.StatePoint(f"{result_path}/statepoint.{statepoint_number}.h5")

    result = {
        "k-eff": sp.keff.nominal_value, 
        "k-eff-std": sp.keff.std_dev,  
        "result_path": result_path
    }

    result.update(run_settings)
    
    return result

print(f"Loading data with {PROCESSES_COUNT} processes...")
with Pool(PROCESSES_COUNT) as p:
    # Loop over each run number and get the run settings as well as the k-effective
    # for result_folder in tqdm(result_folders):
    results = list(tqdm(p.imap(handle_folder, result_folders), total=len(result_folders)))

Importing...
Found 8 result folders
Loading data with 20 processes...


100%|██████████| 8/8 [00:00<00:00, 27.21it/s]


In [4]:
importlib.reload(plot_tools)
importlib.reload(model_tools)

# PLOT SETTINGS:
REACTOR_MODEL = model_tools.SEMIEPITHERMAL_REACTOR
USE_SAMPLED_DATA = True

SAMPLED_STRING = "SAMPLED" if USE_SAMPLED_DATA else "NOT_SAMPLED"

folder_name = f"plots-{REACTOR_MODEL}-{SAMPLED_STRING}"
os.makedirs(folder_name, exist_ok=True)

def get_filename(plot_type: str, value: str):
    return f"{folder_name}/{plot_type}_{value}-{REACTOR_MODEL}-{SAMPLED_STRING}.pdf"

# Create a dataframe from the results
df = pd.DataFrame(results)

# Filters
df = df[df["reactor_model"] == REACTOR_MODEL]
df = df[df["use_sampled_data"] == USE_SAMPLED_DATA]

if df.empty:
    raise Exception("No results found!")

df = data_tools.add_reactivity_to_df(df)

plt.figure()
fig, popt, pcov = plot_tools.plot_histogram_from_df(df=df, data_key="k-eff", 
                                        fit_gaussian=True, decimals=5,
                                        title=f"Distribution of $k_{{eff}}$ for {len(df)} runs, {REACTOR_MODEL.lower()} reactor", x_label="$k_{eff}$", y_label="Count",
                                        filename=get_filename("heatmap", "distribution-k_eff"))

# plt.figure()
# fig, popt, pcov = plot_tools.plot_histogram_from_df(df=df, data_key="rho-pcm", color="mediumseagreen",
#                                         fit_gaussian=True, decimals=0,
#                                         title=fr"Distribution of $\rho$ for {len(df)} runs", x_label=r"$\rho$ [pcm]", y_label="Count",
#                                         filename=get_filename("heatmap", "distribution-rho"))

Exception: No results found!

In [4]:
importlib.reload(plot_tools)
importlib.reload(data_tools)
importlib.reload(model_tools)
import pickle

# Create a dataframe from the results
df = pd.DataFrame(results)

# TMC analysis
for REACTOR_MODEL in [model_tools.SEMIEPITHERMAL_REACTOR]:
    tmc_v1_res = data_tools.perform_TMC_v1(df, REACTOR_MODEL, print_output=True)

    # Save dict to pickle file
    with open(f"R23-tmc_v1_res-{REACTOR_MODEL}.pkl", "wb") as f:
        pickle.dump(tmc_v1_res, f)

    print("")


TMC analysis v1
SEMIEPITHERMAL
Sampling all MTs
Based on 500 runs
 -- k_eff --
k_eff: 		1.277
σ_obs: 		1.70e-03
σ_stat_mean: 	3.15e-04
σ_ND: 		1.67e-03
R_obs: 		133.3 pcm
R_stat_mean: 	24.7 pcm
R_ND: 		131.0 pcm
 -- ρ --
ρ: 		21672.9 pcm
ρ-std: 		102.6 pcm



In [9]:
# TMC analysis 15 april 2023, to get table for paper

importlib.reload(plot_tools)
importlib.reload(data_tools)
importlib.reload(model_tools)
import pickle

# Create a dataframe from the results
df = pd.DataFrame(results)

table_results = []

# TMC analysis
for REACTOR_MODEL in [model_tools.THERMAL_REACTOR, model_tools.SEMIEPITHERMAL_REACTOR, model_tools.EPITHERMAL_REACTOR]:
    tmc_v1_res = data_tools.perform_TMC_v1(df, REACTOR_MODEL, print_output=True)

    if tmc_v1_res is None:
        continue

    u_k = tmc_v1_res["u_k"]
    u_rho = tmc_v1_res["u_rho"]

    # print(tmc_v1_res)
    table_results.append({
        "Reactor model": REACTOR_MODEL.capitalize(),
        "$k_{eff}$": float(f"{u_k.nominal_value:.4f}"),
        "$\sigma_{k_{eff}}$ [pcm]": float(f"{u_k.std_dev*1e5:.1f}"),
        "$\sigma_{k_{eff}}/k_{eff}$ [%]": float(f"{u_k.std_dev/u_k.nominal_value*100:.2f}"),
        r"$\rho$": float(f"{u_rho.nominal_value:.4f}"),
        r"$\sigma_{\rho}$ [pcm]": float(f"{u_rho.std_dev*1e5:.1f}"),
    })

    print(tmc_v1_res)

# df_for_table = pd.DataFrame(table_results)
# table = df_for_table.to_latex(index=False, escape=False)
# print(table)

TMC analysis v1
THERMAL
Sampling all MTs
Based on 8 runs
 -- k_eff --
k_eff: 		1.368
σ_obs: 		1.12e-03
σ_stat_mean: 	3.26e-04
σ_ND: 		1.07e-03
R_obs: 		82.0 pcm
R_stat_mean: 	23.8 pcm
R_ND: 		78.4 pcm
 -- ρ --
ρ: 		26878.4 pcm
ρ-std: 		57.3 pcm
{'sigma_obs': 0.0011209193779402392, 'sigma2_obs': 1.2564602518419328e-06, 'sigma_stat_mean': 0.00032566651473807064, 'sigma2_stat_mean': 1.0605867882164198e-07, 'sigma_ND': 0.0010725677475200766, 'sigma2_ND': 1.1504015730202908e-06, 'k_eff_mean': 1.3675840680585458, 'R_obs': 0.0008196347150574242, 'R_stat_mean': 0.00023813272057226757, 'R_ND': 0.0007842792063545451, 'u_k': 1.3675840680585458+/-0.0010725677475200766, 'u_rho': 0.2687835261055481+/-0.0005734778758193098, 'N_ITERATIONS': 8, 'reactor_model': 'THERMAL'}
No data for SEMIEPITHERMAL
No data for EPITHERMAL


In [16]:
importlib.reload(plot_tools)
importlib.reload(data_tools)
import pickle

# TMC analysis v1 and v3 to see OpenMC discrepancies

# Create a dataframe from the results
df = pd.DataFrame(results)

for REACTOR_MODEL in [model_tools.THERMAL_REACTOR, model_tools.SEMIEPITHERMAL_REACTOR, model_tools.EPITHERMAL_REACTOR]:
    tmc_v1_res = data_tools.perform_TMC_v1(df, REACTOR_MODEL, print_output=False)
    tmc_v3_res = data_tools.perform_TMC_v3(df, REACTOR_MODEL, print_output=False)

    print("")


No data for THERMAL (not sampled)

No data for SEMIEPITHERMAL (not sampled)

No data for EPITHERMAL (not sampled)

